# 안녕하세요. AIFactory입니다 ☺️
이 노트북은 제출을 위한 MNIST Tutorial 가이드라인 입니다.

Colab 사용자는 반드시 구글 드라이브에 사본을 저장을(`파일` -> `Drive에 사본 저장`) 하신 후에, 사본 노트북에서 작업을 해주세요.



# 주의 사항
1. 이 노트북은 추론 자동화 제출을 위한 노트북이니 **학습 코드는 반드시 별도 스크립트에서 진행**해주세요.
2. **Colab 사용자는 권한을 공유 -> 링크가 있는 모든 사용자** 로 설정해주세요.
3. **Colab 사용자는 구글 드라이브 마운트 작업은 하지 말아주세요.** AIF 채점 환경에서 개인 구글 드라이브에 엑세스 할 수 없습니다.
4. 현재 작업중인 경로에서 모델 추론에 **불필요한 파일이 제출되지 않도록 주의**해주세요.
5. 모든 경로는 현재 작업중인 스크립트 파일(task.ipynb)을 기준으로 **상대 경로**로 설정해주세요.
6. AIFactory의 추론 환경은 Colab 환경과 완전히 동일하지 않으니, 아래에서 AIFactory 추론 환경에 설치된 패키지 항목을 참고하신 후 필요한 패키지를 설치해주세요.
7. 채점 환경은 CPU만 사용하여 평가를 진행하니 참고하여 코드를 작성해주세요.

[추론 환경에 설치된 패키지 리스트]
```
tensorflow==2.11.1
keras-cv
torch==1.12.1+cu113 torchvision==0.13.1+cu113 torchaudio==0.12.1
albumentations
transformers
scikit-learn
numpy
Cython
pycocotools
pandas
jupyter
notebook
matplotlib
seaborn
plotly
Pillow
opencv-python
scikit-image
```

# 필요 패키지 설치
현재 작업 환경에 필요한 패키지를 설치합니다.

In [12]:
!pip install -U aifactory
!pip install -U gdown

In [13]:
import random
import numpy as np
import tensorflow as tf
from tensorflow import keras

In [14]:
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

In [ ]:
# 학습 시킨 모델 가중치 파일을 원하는 경로에 미리 준비합니다.
# 이 튜토리얼 코드에서는 사전에 구글 드라이브에 업로드한 가중치 파일을 다운 받는 방식을 사용합니다.

# 가중치 파일은 반드시 다운 받는 방식을 사용 할 필요는 없습니다.
# 현재 스크립트 (task.ipynb) 파일을 기준으로 하위 경로에 가중치 파일을 위치 시킨 후 함께 제출 할 수 있습니다.
# 단, 현재 스크립트 파일 보다 상위 경로는 채점 시 접근이 불가능하니 주의해주세요.
# 자세한 내용은 아래 셀에서 모델 로드 함수를 정의하는 예시를 참고해주세요.

import gdown
gdown.download(
    "https://drive.google.com/file/d/1unELto9YnFkslkc7woweBElQ9yoMnN9Q/view?usp=sharing",
    "./weights.h5", fuzzy=True)

# 모델 로드 함수를 정의해주세요.

- 함수 명칭은 자유롭게 정의해주세요.
- 이 함수가 받는 인자(Arguments)는 없거나, default 값으로 설정 되어야 합니다.
- 모델 로드 함수는 참가자 분들께서 이미 학습 완료한 모델 가중치 파일을 읽어와 메모리에 올려 놓은 후 반환 되어야 합니다.
- 모델 가중치 파일을 현재 노트북 파일 기준 상대 경로로 설정해주세요.


```python
# Example 1
def tf_load_model():
    ...
    model.load_weights('./weights_dir/my_weights.h5')
    return model

# Example 2
def torch_load_model(weights_path='./my_weights.h5'):
    ...
    model.load_state_dict(torch.load(weights_path))
    model.eval()
    return model
```


In [15]:
def custom_load_model():
    # define model
    model = keras.models.Sequential([
        keras.layers.Dense(64, input_dim=28*28, activation='relu'), 
        keras.layers.Dense(32, activation='relu'), 
        keras.layers.Dense(10, activation='softmax')
    ])
    
    # compile model
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    # load weights file
    model.load_weights('./weights.h5')

    return model

# 추론 함수를 정의해주세요.
***입력 규칙*** - 추론 함수는 반드시 **2개의 인자**를 다음과 같이 순서대로 받아야합니다.
- 첫 번째 인자 - AIFactory가 제공하는 평가 입력 데이터(X_test)로, (10000, 28, 28) 형태의 Numpy 배열
- 두 번째 인자 - 가중치를 읽어온 모델

<br>

***출력 규칙*** - 추론 함수는 예측 및 전/후 처리를 통해 반드시 다음 출력 규칙에 맞게 반환 되어야 정상적이 채점이 가능합니다.
- 예측값은 반드시 ***int*** 타입으로 설정해주세요.
- X_test 전체(10,000개)의 예측값을 ***1차원 List 또는 Numpy 배열*** 형태로 반환해주세요.  
    - 추론 함수 반환값 예시: `[0, 3, 1, 2, 5, ... ]` (10,000개의 예측값)

In [16]:
# 추론 함수는 AIF에서 제공하는 (Arg1) X_test의 입력 정보와, (Arg2) 메모리에 올라간 사전 학습 모델을 입력 받습니다.
def custom_inference(X_test, model):
    # 내가 정의한 모델의 입력 형식에 맞게 변환합니다.
    X_test = X_test.reshape(10000, -1) # (10000, 28, 28) -> (10000, 784)

    # 예측    
    prob_pred = model.predict(X_test)

    # 출력 규칙을 따라 예측값을 반환해주세요.
    prob_label = prob_pred.argmax(axis=-1)

    return prob_label


# 제출하기
- aifactory 라이브러리를 통해 함수를 제출합니다.
- ***aifactory.submit_kwargs*** 함수는 4개의 인자를 받습니다.
    - ***model_name*** - 제출시 리더보드에 기록하고자 하는 문구
    - ***key*** - 대회 참가자에게 제공되는 TASK_KEY  
    - ***load_model_fn*** - model load 함수
    - ***inference_fn*** - predict 함수    

<br>

- TASK_KEY 확인 방법
    - https://aifactory.space/competition/detail/2514 페이지 접속
    - 우측 상단 `내정보` -> `태스크 정보` 클릭
    - 1분야 : 이미지 모델 퀘스트에서 `사용자 키 보기` 클릭

In [17]:
import aifactory.score as aif
aif.submit_kwargs(
    model_name='First Submission',
    key='KEY',
    load_model_fn = custom_load_model,
    inference_fn = custom_inference
    )

file : task.py
python


request id : 619 processing...
score = 0.9752
